# LangChain Expression Language

LangChain表达语言，简写为LCEL，是一种声明式的方式来链接LangChain组件。

## 优点
- 一流的流媒体支持 
    当您使用LCEL构建链时，您将获得最佳的首次令牌时间（直到第一个输出块出现所经过的时间）。对于某些链，这意味着例如我们直接从LLM向流媒体输出解析器传输令牌，并且您以与LLM提供商输出原始令牌相同的速度返回解析后的增量输出块。
- 异步支持 
    任何使用LCEL构建的链都可以通过同步API（例如，在原型设计时在你的Jupyter笔记本中）以及异步API（例如，在LangServe服务器中）进行调用。这使得在原型和生产环境中使用相同的代码成为可能，具有出色的性能，并能够在同一服务器上处理许多并发请求。
- 优化的并行执行 
    每当您的LCEL链中有可以并行执行的步骤（例如，如果您从多个检索器获取文档），我们会在同步和异步接口中自动进行，以实现尽可能小的延迟。
- 重试和回退 
    为您的LCEL链的任何部分配置重试和回退。这是使您的链在大规模下更可靠的好方法。我们目前正在努力添加对重试/回退的流媒体支持，这样您就可以在不增加延迟成本的情况下获得额外的可靠性。
- 访问中间结果 
    对于更复杂的链条，通常在最终输出生成之前访问中间步骤的结果非常有用。这可以用于让终端用户知道某些事情正在发生，甚至只是调试你的链条。你可以流式传输中间结果，这在每个LangServe服务器上都可用。
- 输入和输出模式 
    输入和输出模式为每个 LCEL 链提供从链的结构推断出的 Pydantic 和 JSONSchema 模式。这可以用于验证输入和输出，是 LangServe 的一个重要组成部分。
- 无缝LangSmith追踪 
    随着链条变得越来越复杂，了解每一步到底发生了什么变得越来越重要。使用LCEL，所有步骤都会自动记录到LangSmith，以实现最大的可观察性和调试能力。
- 无缝LangServe部署 
    使用LCEL创建的任何链都可以轻松通过LangServe进行部署。

## Runnable接口
    为了尽可能简化创建自定义链条的过程，我们实现了一个“Runnable”协议。许多LangChain组件都实现了Runnable协议，包括聊天模型、LLM、输出解析器、检索器、提示模板等。此外，还有一些有用的基本构件可以用于处理可运行事务，您可以在下面阅读相关信息。
这是一个标准接口，使得定义自定义链以及以标准方式调用它们变得容易。标准接口包括：


todo 解析runnable接口的实现原理